# 1. Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
sns.set_theme(style="whitegrid")

SEED = 42

train_q = pd.read_excel("data/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx")
train_c = pd.read_excel("data/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
test_q = pd.read_excel("data/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
test_c = pd.read_excel("data/TEST/TEST_CATEGORICAL_METADATA.xlsx")

train_combined = pd.merge(train_q, train_c, on='participant_id', how='left').set_index("participant_id")
test_combined = pd.merge(test_q, test_c, on='participant_id', how='left').set_index("participant_id")

labels = pd.read_excel("data/TRAIN/TRAINING_SOLUTIONS.xlsx").set_index("participant_id")

train_combined = train_combined.sort_index()
labels = labels.sort_index()
assert all(train_combined.index == labels.index), "Label IDs do not match train IDs"

# 2. Data Preprocessing

In [ ]:
train_combined.columns

In [ ]:
label_df = labels.copy()

# Plot: ADHD_Outcome
# plt.figure(figsize=(6, 4))
# sns.countplot(x=label_df["ADHD_Outcome"], palette="Set2")
# plt.title("Distribution of ADHD_Outcome")
# plt.xticks([0, 1], ['No ADHD (0)', 'ADHD (1)'])
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()

# # Plot: Sex_F
# plt.figure(figsize=(6, 4))
# sns.countplot(x=label_df["Sex_F"], palette="pastel")
# plt.title("Distribution of Sex_F")
# plt.xticks([0, 1], ['Male (0)', 'Female (1)'])
# plt.ylabel("Count")
# plt.tight_layout()
# plt.show()


## 2.1 Numerical Features Visualization

In [ ]:
numerical_features = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
    'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
    'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
    'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
    'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
    'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan'
]

print(len(numerical_features))

In [ ]:
numerical_features_df = train_combined[numerical_features]

def nan_summary(df, name):
    print(f"---- {name} NAN SUMMARY ----")
    print((df.isnull().sum() / len(df) * 100).sort_values(ascending=False), '\n')

nan_summary(numerical_features_df, "TRAIN NUMERICAL FEATURES")

### 2.1.1 EHQ_EHQ_Total

In [ ]:
feature_1 = 'EHQ_EHQ_Total'

median_value = train_combined[feature_1].median()
train_combined[feature_1] = train_combined[feature_1].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature_1].median()
test_combined[feature_1] = test_combined[feature_1].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

In [ ]:
# plt.figure(figsize=(10, 4))
# sns.boxplot(data=train_combined, x='EHQ_EHQ_Total')
# plt.title("Box Plot of EHQ_|EHQ_Total")
# plt.xlabel("EHQ_EHQ_Total")
# plt.show()

In [ ]:
def categorize_laterality(li):
    if -110 <= li <= -28:
        return "Left-Lateralized"
    elif -28 < li <= 47:
        return 'Middle'
    elif 47 < li <= 110:
        return 'Right-Lateralized'

train_combined['Laterality_Category'] = train_combined['EHQ_EHQ_Total'].apply(categorize_laterality)
test_combined['Laterality_Category'] = test_combined['EHQ_EHQ_Total'].apply(categorize_laterality)

### 2.1.2 ColorVision_CV_Score

In [ ]:
train_combined['ColorVision_CV_Score'].fillna(14, inplace=True)
print("Missing values after fill:", train_combined["ColorVision_CV_Score"].isna().sum())

test_combined['ColorVision_CV_Score'].fillna(14, inplace=True)
print("Missing values after fill:", test_combined["ColorVision_CV_Score"].isna().sum())

### 2.1.3 APQ_P_APQ_P_CP

In [ ]:
nan_count = train_combined['APQ_P_APQ_P_CP'].isna().sum()
print(f"Number of NaN values in APQ_P_APQ_P_CP: {nan_count}")

median_cp = train_combined['APQ_P_APQ_P_CP'].median()
train_combined['APQ_P_APQ_P_CP'] = train_combined['APQ_P_APQ_P_CP'].fillna(median_cp)
print(f"Filled NaNs with median: {median_cp}")

median_test_cp = test_combined['APQ_P_APQ_P_CP'].median()
test_combined['APQ_P_APQ_P_CP'] = test_combined['APQ_P_APQ_P_CP'].fillna(median_test_cp)
print(f"Filled NaNs with median: {median_test_cp}")

### 2.1.4: APQ_P_APQ_P_ID

In [ ]:
feature = 'APQ_P_APQ_P_ID'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in {feature}: {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature].fillna(median_value, inplace=True)
print(f"Filled NaNs in {feature} with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature].fillna(median_test_value, inplace=True)
print(f"Filled NaNs in {feature} with median value: {median_test_value}")

### 2.1.5: APQ_P_APQ_P_INV

In [ ]:
feature = 'APQ_P_APQ_P_INV'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaNs in '{feature}' with median: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaNs in '{feature}' with median: {median_test_value}")

### 2.1.6: APQ_P_APQ_P_OPD

In [ ]:
feature = 'APQ_P_APQ_P_OPD'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaNs in '{feature}' with median: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaNs in '{feature}' with median: {median_test_value}")

### 2.1.7: APQ_P_APQ_P_PM

In [ ]:
feature = 'APQ_P_APQ_P_PM'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaNs in '{feature}' with median: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaNs in '{feature}' with median: {median_test_value}")

### 2.1.8 APQ_P_APQ_P_PP

In [ ]:
feature = 'APQ_P_APQ_P_PP'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaNs in '{feature}' with median: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaNs in '{feature}' with median: {median_test_value}")

### 2.1.9 SDQ_SDQ_Conduct_Problems

In [ ]:
feature = 'SDQ_SDQ_Conduct_Problems'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaNs in '{feature}' with median: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaNs in '{feature}' with median: {median_test_value}")

### 2.1.10 SDQ_SDQ_Difficulties_Total

In [ ]:
feature = 'SDQ_SDQ_Difficulties_Total'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.11: SDQ_SDQ_Emotional_Problems

In [ ]:
feature = 'SDQ_SDQ_Emotional_Problems'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.12: SDQ_SDQ_Externalizing

In [ ]:
feature = 'SDQ_SDQ_Externalizing'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.13: SDQ_SDQ_Generating_Impact

In [ ]:
feature = 'SDQ_SDQ_Generating_Impact'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.14: SDQ_SDQ_Hyperactivity

In [ ]:
feature = 'SDQ_SDQ_Hyperactivity'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.15: SDQ_SDQ_Internalizing

In [ ]:
feature = 'SDQ_SDQ_Internalizing'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.16: SDQ_SDQ_Peer_Problems

In [ ]:
feature = 'SDQ_SDQ_Peer_Problems'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.17: SDQ_SDQ_Prosocial

In [ ]:
feature = 'SDQ_SDQ_Prosocial'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN with median value: {median_test_value}")

### 2.1.18: MRI_Track_Age_at_Scan

In [ ]:
feature = 'MRI_Track_Age_at_Scan'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

In [ ]:
numeric_data = train_combined[numerical_features].copy()
imputer = IterativeImputer(estimator=LassoCV(random_state=SEED), max_iter=100, random_state=SEED)

numeric_data_imputed = pd.DataFrame(
    imputer.fit_transform(numeric_data), 
    columns=numerical_features, 
    index=numeric_data.index
)
train_combined['MRI_Track_Age_at_Scan'] = numeric_data_imputed['MRI_Track_Age_at_Scan']


test_numeric_data = test_combined[numerical_features].copy()
test_numeric_data_imputed = pd.DataFrame(
    imputer.transform(test_numeric_data),
    columns=numerical_features,
    index=test_numeric_data.index
)
test_combined['MRI_Track_Age_at_Scan'] = test_numeric_data_imputed['MRI_Track_Age_at_Scan']

In [ ]:
Q1 = train_combined[feature].quantile(0.25)
Q3 = train_combined[feature].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train_combined[(train_combined[feature] < lower_bound) | (train_combined[feature] > upper_bound)]
print(f"Number of outliers in '{feature}': {len(outliers)}")

plt.figure(figsize=(8, 1.5))
sns.boxplot(x=train_combined[feature], color="lightcoral")
plt.title(f'Boxplot of {feature}')
plt.xlabel(feature)
plt.tight_layout()
plt.show()

In [ ]:
# train_combined.drop(columns=['EHQ_EHQ_Total', 'ColorVision_CV_Score'], inplace=True)
# test_combined.drop(columns=['EHQ_EHQ_Total', 'ColorVision_CV_Score'], inplace=True)

## 2.2 Categorical Features Visualization

In [ ]:
categorical_features = [
    'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
    'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu',
    'Barratt_Barratt_P2_Occ', 'Laterality_Category', 'ColorVision_Level',
    'APQ_CP_is_high'
]

print(len(categorical_features))   

### 2.2.1 Basic_Demos_Enroll_Year

In [ ]:
feature = 'Basic_Demos_Enroll_Year'
nan_count_train = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count_train}")
nan_count_test = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_count_test}")

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette="Set2")
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Compute rare categories from TRAIN only
threshold = 50

# Step 1: Handle rare categories in TRAIN
category_counts = train_combined[feature].value_counts()
rare_categories = category_counts[category_counts < threshold].index
train_combined[feature] = train_combined[feature].apply(
    lambda x: 'Other' if x in rare_categories else str(x)
)

# Step 2: Handle rare + unseen categories in TEST
# Convert to str first so categories match
test_combined[feature] = test_combined[feature].astype(str)

# Map test values to 'Other' if not in train (incl. unseen or rare ones)
allowed_categories = set(train_combined[feature].unique())

test_combined[feature] = test_combined[feature].apply(
    lambda x: x if x in allowed_categories else 'Other'
)

plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette="Set2", order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature} (after grouping rare categories)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
train_combined[feature] = train_combined[feature].replace({4: 0})
test_combined[feature] = test_combined[feature].replace({4: 0})

### 2.2.2: Basic_Demos_Study_Site

In [ ]:
feature = 'Basic_Demos_Study_Site'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_count_test = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_count_test}")

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Combine 2, 3, and 4 into 2
train_combined[feature] = train_combined[feature].replace({3: 2, 4: 2})
test_combined[feature] = test_combined[feature].replace({3: 2, 4: 2})

# Step 2: Handle rare categories
threshold = 25
category_counts = train_combined[feature].value_counts()
rare_categories = category_counts[category_counts < threshold].index

print(f"Rare categories in '{feature}' (fewer than {threshold} instances):")
print(rare_categories)

# Step 3: Replace rare categories with 'Other'
train_combined[feature] = train_combined[feature].apply(
    lambda x: 'Other' if x in rare_categories else str(x)
)
test_combined[feature] = test_combined[feature].apply(
    lambda x: 'Other' if x in rare_categories else str(x)
)

# Step 4: Plot updated TRAIN distribution
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature} (after combining 2/3/4 and cleaning)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2.2.3: PreInt_Demos_Fam_Child_Ethnicity

In [ ]:
feature = 'PreInt_Demos_Fam_Child_Ethnicity'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_count_test = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_count_test}")

In [ ]:
train_combined[feature] = train_combined[feature].fillna(3.0)
test_combined[feature] = test_combined[feature].fillna(3.0)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# combine values 2 and 3
train_combined[feature] = train_combined[feature].replace({3: 2})
test_combined[feature] = test_combined[feature].replace({3: 2})

plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature} (2 and 3 combined)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2.2.4: PreInt_Demos_Fam_Child_Race

In [ ]:
feature = 'PreInt_Demos_Fam_Child_Race'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
train_combined[feature] = train_combined[feature].fillna(10.0)
test_combined[feature] = test_combined[feature].fillna(10.0)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Map values
train_combined[feature] = train_combined[feature].apply(lambda x: "White" if x == 0.0 else "Non-White")
test_combined[feature] = test_combined[feature].apply(lambda x: "White" if x == 0.0 else "Non-White")

# Step 2: Plot
plt.figure(figsize=(6, 4))
sns.countplot(x=train_combined[feature], palette='pastel', 
              order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature} (White vs Non-White)")
plt.tight_layout()
plt.show()

### 2.2.5: MRI_Track_Scan_Location

In [ ]:
feature = 'MRI_Track_Scan_Location'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
train_combined[feature] = train_combined[feature].fillna(4.0)
test_combined[feature] = test_combined[feature].fillna(4.0)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# combine values 1 and 4
train_combined[feature] = train_combined[feature].replace({4: 1})
test_combined[feature] = test_combined[feature].replace({4: 1})

plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2.2.6: Barratt_Barratt_P1_Edu

In [ ]:
feature = 'Barratt_Barratt_P1_Edu'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
train_combined[feature] = train_combined[feature].fillna(18.0)
test_combined[feature] = test_combined[feature].fillna(18.0)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def collapse_edu(value):
    if value == 21.0:
        return "Upper_College"
    elif value == 18.0:
        return "College"
    else:
        return "Under_College"
    
train_combined[feature] = train_combined[feature].apply(collapse_edu)
test_combined[feature] = test_combined[feature].apply(collapse_edu)

In [ ]:
# Plot updated distribution
plt.figure(figsize=(6, 4))
sns.countplot(x=train_combined[feature], palette='Set2', order=train_combined[feature].value_counts().index)
plt.title(f"Collapsed Distribution of {feature}")
plt.tight_layout()
plt.show()

### 2.2.7: Barratt_Barratt_P1_Occ

In [ ]:
feature = 'Barratt_Barratt_P1_Occ'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
median_value = train_combined[feature].median()
train_combined[feature] = train_combined[feature].fillna(median_value)
print(f"Filled NaN in '{feature}' with median value: {median_value}")

median_test_value = test_combined[feature].median()
test_combined[feature] = test_combined[feature].fillna(median_test_value)
print(f"Filled NaN in '{feature}' with median value: {median_test_value}")

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def collapse_occupation(val):
    if val in [0.0, 5.0, 10.0, 15.0]:
        return "Low"
    elif val in [20.0, 25.0]:
        return "LowerMid"
    elif val in [30.0, 35.0, 40.0]:
        return "UpperMid"
    elif val == 45.0:
        return "High"
    else:
        return "Other"  # Safety fallback

feature = "Barratt_Barratt_P1_Occ"

train_combined[feature] = train_combined[feature].apply(collapse_occupation)
test_combined[feature] = test_combined[feature].apply(collapse_occupation)

### 2.2.8: Barratt_Barratt_P2_Edu

In [ ]:
feature = 'Barratt_Barratt_P2_Edu'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
def collapse_edu(value):
    if value == 21.0:
        return "Upper_College"
    elif value == 18.0:
        return "College"
    else:
        return "Under_College"
    
train_combined[feature] = train_combined[feature].apply(collapse_edu)
test_combined[feature] = test_combined[feature].apply(collapse_edu)

In [ ]:
def impute_p2_edu(row):
    if pd.isna(row['Barratt_Barratt_P2_Edu']):
        return row['Barratt_Barratt_P1_Edu']
    return row['Barratt_Barratt_P2_Edu']

train_combined['Barratt_Barratt_P2_Edu'] = train_combined.apply(impute_p2_edu, axis=1)
test_combined['Barratt_Barratt_P2_Edu'] = test_combined.apply(impute_p2_edu, axis=1)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2.2.9: Barratt_Barratt_P2_Occ

In [ ]:
feature = 'Barratt_Barratt_P2_Occ'
nan_count = train_combined[feature].isna().sum()
print(f"Number of NaN rows in feature '{feature}': {nan_count}")

nan_test_count = test_combined[feature].isna().sum()
print(f"Number of NaN rows in test feature '{feature}': {nan_test_count}")

In [ ]:
def collapse_occupation(val):
    if val in [0.0, 5.0, 10.0, 15.0]:
        return "Low"
    elif val in [20.0, 25.0]:
        return "LowerMid"
    elif val in [30.0, 35.0, 40.0]:
        return "UpperMid"
    elif val == 45.0:
        return "High"
    else:
        return "Other"  # Safety fallback

feature = "Barratt_Barratt_P2_Occ"

train_combined[feature] = train_combined[feature].apply(collapse_occupation)
test_combined[feature] = test_combined[feature].apply(collapse_occupation)

In [ ]:
def impute_p2_edu(row):
    if pd.isna(row['Barratt_Barratt_P2_Occ']):
        return row['Barratt_Barratt_P1_Occ']
    return row['Barratt_Barratt_P2_Occ']

train_combined['Barratt_Barratt_P2_Occ'] = train_combined.apply(impute_p2_edu, axis=1)
test_combined['Barratt_Barratt_P2_Occ'] = test_combined.apply(impute_p2_edu, axis=1)

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=train_combined[feature], palette='pastel', order=train_combined[feature].value_counts().index)
plt.title(f"Distribution of {feature}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# train_combined.drop(columns=['MRI_Track_Scan_Location', 'Basic_Demos_Study_Site'], inplace=True)
# test_combined.drop(columns=['MRI_Track_Scan_Location', 'Basic_Demos_Study_Site'], inplace=True)

### 2.2.10 LABEL ENCODING

In [ ]:
label_encode_features = [
    'Basic_Demos_Enroll_Year',
    'Barratt_Barratt_P1_Edu',
    'Barratt_Barratt_P1_Occ',
    'Barratt_Barratt_P2_Edu',
    'Barratt_Barratt_P2_Occ',
    'Laterality_Category'
]



encoders = {}  # Store encoders if needed for inverse_transform later

for feature in label_encode_features:
    le = LabelEncoder()
    train_combined[feature] = le.fit_transform(train_combined[feature])
    test_combined[feature] = le.transform(test_combined[feature])
    encoders[feature] = le  # Save encoder

In [ ]:
# Updated binary map: White = 1, Non-White = 0
binary_map = {
    'White': 1,
    'Non-White': 0,
}

# Apply to 'PreInt_Demos_Fam_Child_Race'
train_combined['PreInt_Demos_Fam_Child_Race'] = train_combined['PreInt_Demos_Fam_Child_Race'].map(binary_map)
test_combined['PreInt_Demos_Fam_Child_Race'] = test_combined['PreInt_Demos_Fam_Child_Race'].map(binary_map)

# 3. Final

In [ ]:
train_combined.columns

In [ ]:
test_combined

In [ ]:
for feature in train_combined.columns:
    num_missing = train_combined[feature].isna().sum()
    print(f"{feature}: {num_missing} missing values")

In [ ]:
for feature in test_combined.columns:
    num_missing = test_combined[feature].isna().sum()
    print(f"{feature}: {num_missing} missing values")

In [ ]:
# Reset index to include participant_id as a column
train_combined.reset_index(inplace=True)
test_combined.reset_index(inplace=True)

train_combined.to_csv('train_processed.csv', index=False)
test_combined.to_csv('test_processed.csv', index=False)

print("Complete!")